In [1]:
from telethon import TelegramClient
import pandas as pd
import json 
from pathlib import Path
import json
import os
from dotenv import load_dotenv
import asyncio
from asyncio import TimeoutError
from pprint import pprint
import re
print(load_dotenv())
api_id = os.getenv("TELEGRAM_API_ID")
api_hash = os.getenv("TELEGRAM_API_HASH")
phone_num = os.getenv("PHONE_NUM")
session_name = "GMGN_ATH_Group"
from telethon.errors import ChatAdminRequiredError, ChannelPrivateError
from telethon.sessions import MemorySession
from telethon.sessions import StringSession

True


In [2]:
# Add this function to save the session string
async def save_session():
    client = TelegramClient(StringSession(), api_id, api_hash)
    await client.start()
    session_string = client.session.save()
    with open('session.txt', 'w') as f:
        f.write(session_string)
    await client.disconnect()
    return session_string

# Only run this once to create the session
await save_session()

Signed in successfully as DM; remember to not break the ToS or you will risk an account ban!


'1AZWarzoBu4-2QVET_zbIaaC30-jpJxBNf6CHxjgTYwTm1QMWOjCiptITvW8gozFL2RMGCJkB1eRAb2y65HqmsahMhGKBAA9ec8gqnV-qt6WXVga1f62GuxanSHUPuGyGcDBDmaHd2ll8riwqiDAi1OuZRsxF9NhQWJUZEcUkvHhpNvKeVEDrcO9nhtfW6lvBKIJfrwoBc8tmE4Dlahk8nebAuTReJMwvCSTPvXlQSafZGil3dlruUaQXN-KI6VyzTuu3h7n1OdXkrxIqBSwhLTvJ1aooRqFX_PoE04WYpJ-ppZcVX6PsdPhcoshYKSVOZ-WI0JRpPkNE9RL3w2MklmnZOEqzDHo='

In [2]:
from telethon.tl.functions.channels import GetForumTopicsRequest
from telethon.sessions import StringSession

async def main(chat_name, limit, topic_id=None):
    # Load existing session
    try:
        with open('session.txt', 'r') as f:
            session_string = f.read()
        client = TelegramClient(StringSession(session_string), api_id, api_hash)
    except FileNotFoundError:
        print("No saved session found, using memory session")
        client = TelegramClient(MemorySession(), api_id, api_hash)
    
    try:
        print("Starting client...")
        await client.start()
        
        try:
            print(f"Attempting to connect to chat: {chat_name}")
            # Get chat info 
            chat_info = await client.get_entity(chat_name)
            print(f"Successfully connected to chat: {chat_info.title}")
            
            # Get topic title if topic_id is provided
            topic_title = None
            if topic_id:
                result = await client(GetForumTopicsRequest(
                    channel=chat_info,
                    offset_date=0,
                    offset_id=0,
                    offset_topic=0,
                    limit=100
                ))
                topic_title = next((topic.title for topic in result.topics 
                                  if hasattr(topic, 'id') and topic.id == topic_id), "Unknown Topic")
                print(f"Fetching messages from topic: {topic_title} (ID: {topic_id})")
            
            # Get all the messages from the specific topic
            print("Fetching messages...")
            messages = await client.get_messages(
                entity=chat_info, 
                limit=limit,
                reply_to=topic_id
            )
            print(f"Retrieved {len(messages)} messages")
            return {
                "messages": messages, 
                "channel": chat_info,
                "topic_title": topic_title
            }
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None
            
    finally:
        print("Disconnecting client...")
        await client.disconnect()

async def main_with_timeout(chat_name, limit, topic_id=None, timeout=120):
    try:
        result = await asyncio.wait_for(main(chat_name, limit, topic_id), timeout=timeout)
        return result
    except TimeoutError:
        print(f"Operation timed out after {timeout} seconds")
        return None

In [3]:
print('''
ID: 1115814 - Title: KOL FOMO
ID: 1115806 - Title: Smart Money FOMO
ID: 1 - Title: General
ID: 1383119 - Title: Hall of Fame
ID: 1152511 - Title: PUMP FDV Surge
ID: 1152525 - Title: Solana FDV Surge
ID: 1268284 - Title: Solana New Pool Alert
ID: 1273756 - Title: Pump King of the hill（KOTH）
ID: 1153178 - Title: Heavy Bought
ID: 1152477 - Title: DEV Burnt Alert
ID: 1272338 - Title: Pump Completed Alert
ID: 1115976 - Title: ATH Price
ID: 1436168 - Title: Pump DEV Activity
ID: 1120024 - Title: Update DEX Screener Social
ID: 1115997 - Title: Sniper New
ID: 1116022 - Title: Pump Update DEX Screener Social
ID: 1115926 - Title: CTO
ID: 1272382 - Title: Sol Lp Burn
ID: 1115948 - Title: Advertised on DEXScreener
ID: 167118 - Title: Pump DEV Activity
''')



ID: 1115814 - Title: KOL FOMO
ID: 1115806 - Title: Smart Money FOMO
ID: 1 - Title: General
ID: 1383119 - Title: Hall of Fame
ID: 1152511 - Title: PUMP FDV Surge
ID: 1152525 - Title: Solana FDV Surge
ID: 1268284 - Title: Solana New Pool Alert
ID: 1273756 - Title: Pump King of the hill（KOTH）
ID: 1153178 - Title: Heavy Bought
ID: 1152477 - Title: DEV Burnt Alert
ID: 1272338 - Title: Pump Completed Alert
ID: 1115976 - Title: ATH Price
ID: 1436168 - Title: Pump DEV Activity
ID: 1120024 - Title: Update DEX Screener Social
ID: 1115997 - Title: Sniper New
ID: 1116022 - Title: Pump Update DEX Screener Social
ID: 1115926 - Title: CTO
ID: 1272382 - Title: Sol Lp Burn
ID: 1115948 - Title: Advertised on DEXScreener
ID: 167118 - Title: Pump DEV Activity



ATH PRICE

In [4]:
# Test the connection with a smaller limit first
chat_name = "gmgnsignals"
topic_id = 1115976  # This is the ID for ATH Price topic
limit = 2

print("Starting main function...")
ATH_price_result = await main_with_timeout(chat_name, limit, topic_id)
if ATH_price_result and ATH_price_result["messages"]:
    print(f"\nMessages from topic: {ATH_price_result['topic_title']}")
    print("=" * 50)
    print(f"Retrieved {len(ATH_price_result['messages'])} messages:")
    
    # Loop through all messages in reverse order (oldest to newest)
    for i, message in enumerate(reversed(ATH_price_result["messages"]), 1):
        print(f"\nMessage {i} of {len(ATH_price_result['messages'])}:")
        print("-" * 50)
        print(message.text)
        print("-" * 50)
else:
    print("No messages retrieved")

Starting main function...
Starting client...


Attempting to connect to chat: gmgnsignals
Successfully connected to chat: GMGN Featured Signals - SOL
Fetching messages from topic: ATH Price (ID: 1115976)
Fetching messages...
Retrieved 2 messages
Disconnecting client...

Messages from topic: ATH Price
Retrieved 2 messages:

Message 1 of 2:
--------------------------------------------------
**💊ATH Price💊**

**$PEPE**(Narcos Hippo Shot By Police)
`Ex1Y91GMvmivY5px7Xp4RaMy3cBxxuAJbkF4ZTk4pump`

📈 5m | 1h | 6h: **91.15%** | **61.49%** | **9119.3%**
🎲 5m TXs/Vol: **190**/**$105.9K**
💡 MCP: **$563.2K**
💧 Liq: **152.51** **SOL** ($64K 🔥100%)
👥 Holder: **436**
🕒 Open: **1h** **ago**

✅ NoMint / ✅Blacklist / ✅Burnt
✅TOP 10: **29.62%**

⏳ DEV: [🚨 Sell All](https://gmgn.ai/sol/address/DjHDuHT9wraX4jmhhfrvUmy6eJS91sGGtLHBBehpK9bV)
👨‍🍳 DEV Burnt烧币: -

Backup BOT: [US](https://t.me/US_GMGNBOT?start=Ex1Y91GMvmivY5px7Xp4RaMy3cBxxuAJbkF4ZTk4pump) | [01](https://t.me/GMGN_sol_bot?start=Ex1Y91GMvmivY5px7Xp4RaMy3cBxxuAJbkF4ZTk4pump) | [02](https://t.me

In [19]:
import re
import pandas as pd

def parse_ath_price_messages(messages):
    data = []
    
    for message in messages:
        message_data = {
            'name': None,
            'token': None,
            '5m_change': None,
            '1h_change': None,
            '6h_change': None,
            '5m_transactions': None,
            '5m_volume': None,
            'mcp': None,
            'liquidity_sol': None,
            'holders': None,
            'open_time': None,
            'top10_holders': None,
            'dev_hold_%': None,
            'dev_burnt': None,
            'telegram': None
        }
        
        # Split message text into lines
        lines = message.text.split('\n')
        
        # Process each line
        for line in lines:
            # Extract name
            if '**$' in line:
                name_match = re.search(r'\((.*?)\)', line)
                if name_match:
                    message_data['name'] = name_match.group(1).strip()
            
            # Extract token address
            if line.strip().startswith('`') and line.strip().endswith('`'):
                message_data['token'] = line.strip('`')
            
            # Extract percentage changes
            if '📈 5m | 1h | 6h:' in line:
                percentages = re.findall(r'[-\d.]+%', line)
                if len(percentages) == 3:
                    message_data['5m_change'] = percentages[0]
                    message_data['1h_change'] = percentages[1]
                    message_data['6h_change'] = percentages[2]
            
            # Extract 5m Transactions and Volume
            if '🎲 5m TXs/Vol:' in line:
                tx_vol = re.search(r'5m TXs/Vol:\s*\*+(\d+)\*+/\*+\$([\d.,]+K)\*+', line)
                if tx_vol:
                    message_data['5m_transactions'] = tx_vol.group(1)
                    message_data['5m_volume'] = tx_vol.group(2)
            
            # Extract MCP
            if '💡 MCP:' in line:
                mcp = re.search(r'MCP:\s*\*+\$([\d.,]+K)\*+', line)
                if mcp:
                    message_data['mcp'] = mcp.group(1)
            
            # Extract Liquidity SOL
            if '💧 Liq:' in line:
                liq = re.search(r'Liq:\s*\*+([\d.]+)\*+\s*\*+SOL\*+', line)
                if liq:
                    message_data['liquidity_sol'] = liq.group(1)
            
            # Extract Holders
            if '👥 Holder:' in line:
                holder = re.search(r'Holder:\s*\*+(\d+)\*+', line)
                if holder:
                    message_data['holders'] = holder.group(1)
            
            # Extract Open Time
            if '🕒 Open:' in line:
                open_time = re.search(r'Open:\s*\*+([\w\d]+)\*+\s*\*+ago\*+', line)
                if open_time:
                    message_data['open_time'] = open_time.group(1)
            
            # Extract Top 10 Holders
            if '✅TOP 10:' in line:
                top10 = re.search(r'TOP 10:\s*\*+([\d.]+)%\*+', line)
                if top10:
                    message_data['top10_holders'] = top10.group(1) + '%'
            
            # Extract DEV Status
            if '⏳ DEV:' in line:
                message_data['dev_hold_%'] = "0%"
            
            # Extract DEV Burnt
            if 'DEV Burnt' in line:
                burnt = re.search(r'DEV Burnt.*?:\s*\*+(.+?)\*+', line)
                if burnt and burnt.group(1).strip() != '-':
                    message_data['dev_burnt'] = burnt.group(1).strip()
                else:
                    message_data['dev_burnt'] = '-'
            
            # Extract Telegram Links
            if 'Backup BOT:' in line:
                telegram_links = re.findall(r'\[.*?\]\((https?://t\.me/.*?)\)', line)
                message_data['telegram'] = ', '.join(telegram_links) if telegram_links else None
        
        data.append(message_data)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Reorder columns with all desired fields
    desired_columns = [
        'name', 'token', '5m_change', '1h_change', '6h_change', 
        '5m_transactions', '5m_volume', 'mcp', 'liquidity_sol', 
        'holders', 'open_time', 'top10_holders', 'dev_hold_%', 
        'dev_burnt', 'telegram'
    ]
    
    # Only include columns that exist in the DataFrame
    columns = [col for col in desired_columns if col in df.columns]
    
    return df[columns]

# Usage:
if ATH_price_result and ATH_price_result["messages"]:
    ath_price_df = parse_ath_price_messages(ATH_price_result["messages"])
ath_price_df

,name,token,5m_change,1h_change,6h_change,5m_transactions,5m_volume,mcp,liquidity_sol,holders,open_time,top10_holders,dev_hold_%,dev_burnt,telegram
0,VIBE,4iHFhUSVH5ddaxFTfE497ufKjW5B4rrGFMPpnMb7pump,34.37%,513.1%,513.1%,52,8.2K,37.6K,44.52,141,43min,23.17%,0%,-,https://t.me/US_GMGNBOT?start=4iHFhUSVH5ddaxFT...
1,Narcos Hippo Shot By Police,Ex1Y91GMvmivY5px7Xp4RaMy3cBxxuAJbkF4ZTk4pump,91.15%,61.49%,9119.3%,190,105.9K,563.2K,152.51,436,1h,29.62%,0%,-,https://t.me/US_GMGNBOT?start=Ex1Y91GMvmivY5px...


SOLANA FDV SURGE

In [21]:
# Test the connection with a smaller limit first
chat_name = "gmgnsignals"
topic_id = 1152525  # This is the ID for ATH Price topic
limit = 2

print("Starting main function...")
FDV_surge_result = await main_with_timeout(chat_name, limit, topic_id)
if FDV_surge_result and FDV_surge_result["messages"]:
    print(f"\nMessages from topic: {FDV_surge_result['topic_title']}")
    print("=" * 50)
    print(f"Retrieved {len(FDV_surge_result['messages'])} messages:")
    
    # Loop through all messages in reverse order (oldest to newest)
    for i, message in enumerate(reversed(FDV_surge_result["messages"]), 1):
        print(f"\nMessage {i} of {len(FDV_surge_result['messages'])}:")
        print("-" * 50)
        print(message.text)
        print("-" * 50)
else:
    print("No messages retrieved")

Starting main function...
Starting client...


Attempting to connect to chat: gmgnsignals
Successfully connected to chat: GMGN Featured Signals - SOL
Fetching messages from topic: Solana FDV Surge (ID: 1152525)
Fetching messages...
Retrieved 2 messages
Disconnecting client...

Messages from topic: Solana FDV Surge
Retrieved 2 messages:

Message 1 of 2:
--------------------------------------------------
**Solana 市值飙升 FDV Surge Alert**

**FDV in 5 min 🟢+$136.8K(+158.6%)**

**$bloop**(bloop)
`7yK3PXJnRHW6QXNRwDXU1qEGHQAcDe2RP1Xx3MWbpump`

📈 5m | 1h | 6h: **158.6%** | **3676.6%** | **3676.6%**
🎲 5m TXs/Vol: **504**/**$451.4K**
💡 MCP: **$223.1K**
💧 Liq: **79.01** **SOL** ($33.2K 🔥100%)
👥 Holder: **234**
🕒 Open: **3min** **ago**

✅ NoMint / ✅Blacklist / ✅Burnt
✅TOP 10: **25.43%**

⏳ DEV: [0.71% HOLD](https://gmgn.ai/sol/address/DC7e51qzR1q3G7RepLi4RJvmEwKKRTSjbPx1ukAC84Us)
👨‍🍳 DEV Burnt烧币: -

Backup BOT: [US](https://t.me/US_GMGNBOT?start=7yK3PXJnRHW6QXNRwDXU1qEGHQAcDe2RP1Xx3MWbpump) | [01](https://t.me/GMGN_sol_bot?start=7yK3PXJnRHW6QXN

In [27]:
import re
import pandas as pd

def parse_fdv_surge_messages(messages):
    data = []
    
    for message in messages:
        message_data = {
            'name': None,
            'token': None,
            'fdv_surge_amount': None,
            'fdv_surge_percent': None,
            '5m_change': None,
            '1h_change': None,
            '6h_change': None,
            '5m_transactions': None,
            '5m_volume': None,
            'mcp': None,
            'liquidity_sol': None,
            'holders': None,
            'open_time': None,
            'top10_holders': None,
            'dev_hold_%': None,
            'dev_burnt': None,
            'telegram': None
        }
        
        # Split message text into lines
        lines = message.text.split('\n')
        
        # Process each line
        for line in lines:
            # Extract FDV surge amount and percentage
            if 'FDV in 5 min' in line:
                fdv_amount = re.search(r'FDV in 5 min.*?\$([\d.,K]+)', line, re.IGNORECASE)
                fdv_percent = re.search(r'\(([\+\-]?\d+\.?\d*%)\)', line)
                if fdv_amount:
                    message_data['fdv_surge_amount'] = fdv_amount.group(1)
                if fdv_percent:
                    message_data['fdv_surge_percent'] = fdv_percent.group(1)
            
            # Extract token name
            if '**$' in line:
                name_match = re.search(r'\((.*?)\)', line)
                if name_match:
                    message_data['name'] = name_match.group(1).strip()
            
            # Extract token address
            if line.strip().startswith('`') and line.strip().endswith('`'):
                message_data['token'] = line.strip('`')
            
            # Extract percentage changes
            if '📈 5m | 1h | 6h:' in line:
                percentages = re.findall(r'[-\d.]+%', line)
                if len(percentages) == 3:
                    message_data['5m_change'] = percentages[0]
                    message_data['1h_change'] = percentages[1]
                    message_data['6h_change'] = percentages[2]
            
            # Extract 5m Transactions and Volume
            if '🎲 5m TXs/Vol:' in line:
                tx_vol = re.search(r'5m TXs/Vol:\s*\*+([\d.,K]+)\*+/\*+\$([\d.,K]+)\*+', line)
                if tx_vol:
                    message_data['5m_transactions'] = tx_vol.group(1)
                    message_data['5m_volume'] = tx_vol.group(2)
            
            # Extract MCP
            if '💡 MCP:' in line:
                mcp = re.search(r'MCP:\s*\*+\$?([\d.,K]+M?)\*+', line)
                if mcp:
                    message_data['mcp'] = mcp.group(1)
            
            # Extract Liquidity SOL
            if '💧 Liq:' in line:
                liq = re.search(r'Liq:\s*\*+([\d.]+)\*+\s*\*+SOL\*+', line)
                if liq:
                    message_data['liquidity_sol'] = liq.group(1)
            
            # Extract Holders
            if '👥 Holder:' in line:
                holders = re.search(r'Holder:\s*\*+([\d,]+)\*+', line)
                if holders:
                    message_data['holders'] = holders.group(1)
            
            # Extract Open Time
            if '🕒 Open:' in line:
                open_time = re.search(r'Open:\s*\*+([\w\d]+)\*+\s*\*+ago\*+', line)
                if open_time:
                    message_data['open_time'] = open_time.group(1)
            
            # Extract Top 10 Holders
            if 'TOP 10:' in line:
                top10 = re.search(r'TOP 10:\s*\*+([\d.]+%)\*+', line)
                if top10:
                    message_data['top10_holders'] = top10.group(1)
            
            # Extract DEV Status
            if '⏳ DEV:' in line:
                dev_status = re.search(r'\[(.*?)\]', line)
                if dev_status:
                    status = dev_status.group(1)
                    if status == "🚨 Sell All":
                        message_data['dev_hold_%'] = "0%"
                    elif "HOLD" in status.upper():
                        percentage = re.search(r'([\d.]+)%', status)
                        if percentage:
                            message_data['dev_hold_%'] = f"{percentage.group(1)}%"
                    else:
                        message_data['dev_hold_%'] = status
            
            # Extract DEV Burnt
            if 'DEV Burnt' in line:
                burnt = re.search(r'DEV Burnt.*?:\s*\*+([\w\-/]+)\*+', line)
                if burnt and burnt.group(1).strip() != '-':
                    message_data['dev_burnt'] = burnt.group(1).strip()
                else:
                    message_data['dev_burnt'] = '-'
            
            # Extract Telegram Links
            if '[✈️ Telegram]' in line:
                telegram = re.findall(r'\((https?://t\.me/.*?)\)', line)
                if telegram:
                    message_data['telegram'] = ', '.join(telegram)
        
        data.append(message_data)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Reorder columns with all desired fields
    desired_columns = [
        'name', 'token', 'fdv_surge_amount', 'fdv_surge_percent',
        '5m_change', '1h_change', '6h_change', '5m_transactions', 
        '5m_volume', 'mcp', 'liquidity_sol', 'holders', 'open_time', 
        'top10_holders', 'dev_hold_%', 'dev_burnt', 'telegram'
    ]
    
    # Only include columns that exist in the DataFrame
    columns = [col for col in desired_columns if col in df.columns]
    
    return df[columns]

# Usage:
if FDV_surge_result and FDV_surge_result["messages"]:
    fdv_surge_df = parse_fdv_surge_messages(FDV_surge_result["messages"])
fdv_surge_df

,name,token,fdv_surge_amount,fdv_surge_percent,5m_change,1h_change,6h_change,5m_transactions,5m_volume,mcp,liquidity_sol,holders,open_time,top10_holders,dev_hold_%,dev_burnt,telegram
0,Coinbase Mascot,3FBowPDWesPkf9M4SDHom4ANrkowkLtZrsKod3Ezpump,601.5K,+33.32%,None,None,None,515,186.4K,2.4M,476.1,3151,3h,12.79%,0%,-,https://t.me/ClarkCB
1,bloop,7yK3PXJnRHW6QXNRwDXU1qEGHQAcDe2RP1Xx3MWbpump,136.8K,+158.6%,158.6%,3676.6%,3676.6%,504,451.4K,223.1K,79.01,234,3min,25.43%,0.71%,-,https://t.me/+9BlvYe2dpPJlZTZk


In [92]:
from telethon.tl.types import Message, Channel, ForumTopicDeleted
from telethon.tl.functions.channels import GetForumTopicsRequest
from telethon.sessions import StringSession

async def get_topics(chat_name):
    # Load existing session
    try:
        with open('session.txt', 'r') as f:
            session_string = f.read()
        client = TelegramClient(StringSession(session_string), api_id, api_hash)
    except FileNotFoundError:
        print("No saved session found, using memory session")
        client = TelegramClient(MemorySession(), api_id, api_hash)
    
    try:
        print("Starting client...")
        await client.start()
        
        try:
            print(f"Attempting to connect to chat: {chat_name}")
            chat = await client.get_entity(chat_name)
            
            if not isinstance(chat, Channel):
                print("This is not a channel/group")
                return None
                
            print("Fetching topics...")
            result = await client(GetForumTopicsRequest(
                channel=chat,
                offset_date=0,
                offset_id=0,
                offset_topic=0,
                limit=100
            ))
            
            topics = []
            for topic in result.topics:
                # Skip deleted topics
                if isinstance(topic, ForumTopicDeleted):
                    continue
                    
                topic_data = {
                    'id': topic.id,
                    'title': topic.title if hasattr(topic, 'title') else 'Untitled',
                    'top_message': topic.top_message if hasattr(topic, 'top_message') else None
                }
                topics.append(topic_data)
            
            print(f"\nFound {len(topics)} active topics:")
            for topic in topics:
                print(f"ID: {topic['id']} - Title: {topic['title']}")
            
            return topics
            
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            return None
            
    finally:
        print("Disconnecting client...")
        await client.disconnect()

# Use it like this:
chat_name = "gmgnsignals"
topics = await get_topics(chat_name)

# Save topics to file if needed
if topics:
    with open('telegram_topics.json', 'w') as f:
        json.dump(topics, f, indent=2)
    print("\nTopics saved to telegram_topics.json")

SyntaxError: incomplete input (3517003661.py, line 1)